In [1]:
from coffea import hist
from coffea.analysis_objects import JaggedCandidateArray
import coffea.processor as processor
import uproot

import numpy as np
np.seterr(divide='ignore', invalid='ignore', over='ignore')
import matplotlib.pyplot as plt
from FireHydrant.Tools.metfilter import MetFilters
from FireHydrant.Tools.correction import get_pu_weights_function, get_ttbar_weight, get_nlo_weight_function

# import pandas as pd
import awkward
from uproot_methods import TLorentzVectorArray
from FireHydrant.Tools.trigger import Triggers

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [9]:
import json
import os
from os.path import join
datasets_=json.load(open(join(os.getenv('FH_BASE'), 'Notebooks/MC/Samples/skimmedbackgrounds_v2.json')))
datasets = {}
for group in datasets_:
    for tag in datasets_[group]:
        datasets[tag] = datasets_[group][tag]

In [5]:
class LeptonJetProcessor(processor.ProcessorABC):
    def __init__(self):
        dataset_axis = hist.Cat('dataset', 'backgrounds')
        self._accumulator = processor.dict_accumulator({
            'run': processor.column_accumulator(np.zeros(shape=(0,))),
            'lumi': processor.column_accumulator(np.zeros(shape=(0,))),
            'event': processor.column_accumulator(np.zeros(shape=(0,))),
        })
        
    @property
    def accumulator(self):
        return self._accumulator
    
    def process(self, df):
        output = self.accumulator.identity()
        if df.size==0: return output
        
        dataset = df['dataset']
        run = df['run']
        lumi = df['lumi']
        event = df['event']
        
        output['run'] += processor.column_accumulator(run)
        output['lumi'] += processor.column_accumulator(lumi)
        output['event'] += processor.column_accumulator(event)
        
        return output
    
    def postprocess(self, accumulator):
        return accumulator

In [12]:
outputs={}
for k in datasets:
    outputs[k] = processor.run_uproot_job({k: datasets[k]},
                                  treename='ffNtuplizer/ffNtuple',
                                  processor_instance=LeptonJetProcessor(),
                                  executor=processor.futures_executor,
                                  executor_args=dict(workers=12, flatten=True),
                                  chunksize=500000,
                                 )

Processing: 100%|██████████| 6/6 [00:02<00:00,  2.99items/s]


In [14]:
for k, output in outputs.items():
    runs, lumis, events = output['run'].value.astype(int), output['lumi'].value.astype(int), output['event'].value.astype(int)
    print(f'\n{k} [{len(runs)}]')
    if runs.size==0: continue
    for r,l,e in np.nditer([runs, lumis, events]):
        print('{}:{}:{}'.format(r, l, e))


QCD_Pt-50to80 [5]
1:1079:14016546
1:1221:15865438
1:1391:18071594
1:2458:31945794
1:2735:35542474

QCD_Pt-80to120 [3]
1:2749:67086890
1:5764:140637777
1:25863:631012355

QCD_Pt-120to170 [6]
1:3506:89664395
1:7203:184204313
1:8190:209438390
1:14873:380355483
1:14956:382493885
1:14778:377927487

QCD_Pt-170to300 [13]
1:7730:183572284
1:16777:398434616
1:8694:206478418
1:9481:225155007
1:11351:269572841
1:18855:447791806
1:13951:331318599
1:14346:340696289
1:16796:398902850
1:17563:417120257
1:17934:425930052
1:21697:515287201
1:22080:524385010

QCD_Pt-300to470 [20]
1:2569:19380067
1:6137:46305630
1:1629:12285834
1:42664:321939141
1:7450:56212638
1:40996:309354726
1:9949:75074634
1:6354:47944131
1:35168:265370738
1:40065:302326773
1:14988:113096548
1:14760:111375661
1:23076:174124438
1:28449:214674679
1:30581:230757167
1:17536:132326412
1:17352:130930689
1:19959:150606896
1:24646:185976760
1:38328:289215690

QCD_Pt-470to600 [18]
1:367:7017271
1:1152:22024312
1:1100:21035635
1:3415:6533648